# 🚀 VideoWise Backend - Complete Colab Setup

This notebook runs the complete VideoWise backend API server with video summarization.

## ⚠️ IMPORTANT: Run ALL cells in order!

1. **Enable GPU**: Runtime → Change runtime type → GPU (T4) → Save
2. Run cells 1-5 in order (don't skip any!)
3. Upload `app.py` and `video_processor.py` when prompted
4. Copy the public URL and use it in your frontend `.env`
5. **Keep the last cell (Step 5) running!** - This is the server


## 📦 Step 1: Install Dependencies


In [1]:
# Install all required packages
!pip install -q flask flask-cors flask-jwt-extended werkzeug python-dotenv
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers accelerate
!pip install -q opencv-python pillow
!pip install -q librosa soundfile
!pip install -q easyocr
!pip install -q moviepy
!pip install -q sentencepiece protobuf
!pip install -q groq google-generativeai

print("✅ All dependencies installed!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 4.5 MB/s eta 0:00:00
✅ All dependencies installed!


## 🔑 Step 2: Configure API Keys


In [ ]:
# Set your API keys here
import os

# JWT Secret Key
os.environ['JWT_SECRET_KEY'] = 'dev-secret-key-change-this-12345'

# LLM API Keys (get from https://console.groq.com/ and https://makersuite.google.com/app/apikey)
os.environ['GROQ_API_KEY'] =   # 👈 ADD YOUR GROQ API KEY HERE
os.environ['GEMINI_API_KEY'] =   # 👈 ADD YOUR GEMINI API KEY HERE

# Other settings
os.environ['UPLOAD_FOLDER'] = '/content/uploads'
os.environ['DB_PATH'] = '/content/videowise.db'
os.environ['FLASK_ENV'] = 'development'

print("✅ Environment variables set!")
print("\n⚠️  Don't forget to add your GROQ_API_KEY and GEMINI_API_KEY above!")


✅ Environment variables set!

⚠️  Don't forget to add your GROQ_API_KEY and GEMINI_API_KEY above!


## 📁 Step 3: Upload Backend Files

**CRITICAL**: You must upload `app.py` and `video_processor.py` from your local `backend/` folder


In [3]:
# Create directories
!mkdir -p /content/uploads

# Check if files exist
import os

if os.path.exists('/content/app.py') and os.path.exists('/content/video_processor.py'):
    print("✅ Backend files found!")
else:
    print("⚠️  Backend files NOT found!")
    print("\n📋 To upload files:")
    print("   1. Click the folder icon (📁) on the left sidebar")
    print("   2. Click 'Upload to session storage'")
    print("   3. Upload app.py and video_processor.py from your local backend/ folder")
    print("   4. Re-run this cell to verify")


⚠️  Backend files NOT found!

📋 To upload files:
   1. Click the folder icon (📁) on the left sidebar
   2. Click 'Upload to session storage'
   3. Upload app.py and video_processor.py from your local backend/ folder
   4. Re-run this cell to verify


## 📤 Alternative: Upload Files Using Code

If you prefer, use this cell to upload files (run it and select files when prompted):


In [4]:
# Upload files using Colab's file picker
from google.colab import files
import shutil
import os

print("📤 Step 1: Upload app.py")
print("   (A file picker will appear - select app.py from your local backend/ folder)")
uploaded = files.upload()

# Move to /content/
for filename in uploaded.keys():
    if filename.endswith('.py'):
        shutil.move(filename, f'/content/{filename}')
        print(f"✅ Uploaded and moved {filename} to /content/")

print("\n📤 Step 2: Upload video_processor.py")
print("   (Select video_processor.py from your local backend/ folder)")
uploaded = files.upload()

for filename in uploaded.keys():
    if filename.endswith('.py'):
        shutil.move(filename, f'/content/{filename}')
        print(f"✅ Uploaded and moved {filename} to /content/")

# Verify
print("\n📋 Verifying files...")
if os.path.exists('/content/app.py') and os.path.exists('/content/video_processor.py'):
    print("✅ Both files are ready!")
else:
    print("⚠️  Some files are missing. Please upload them again.")


📤 Step 1: Upload app.py
   (A file picker will appear - select app.py from your local backend/ folder)


Saving video_processor.py to video_processor.py
Saving app.py to app.py
✅ Uploaded and moved video_processor.py to /content/
✅ Uploaded and moved app.py to /content/

📤 Step 2: Upload video_processor.py
   (Select video_processor.py from your local backend/ folder)



📋 Verifying files...
✅ Both files are ready!


## 🌐 Step 4: Setup Cloudflare Tunnel (Free, No Signup!)

This creates a public URL so your frontend can connect to the backend.


In [5]:
# Setup Cloudflare Tunnel (free, no authentication needed!)
print("🌐 Setting up Cloudflare Tunnel...")
print("="*60)

# Download cloudflared
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

import subprocess
import threading
import time
import re

public_url = None
tunnel_process = None

def start_cloudflare_tunnel():
    """Start Cloudflare tunnel and capture URL"""
    global public_url, tunnel_process
    try:
        # Start cloudflared tunnel
        process = subprocess.Popen(
            ['./cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1
        )
        tunnel_process = process

        # Read output to find URL
        for line in process.stdout:
            print(line, end='')
            # Look for URL pattern
            url_match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if url_match:
                public_url = url_match.group(0)
                break

        return process
    except Exception as e:
        print(f"Error: {e}")
        return None

# Start tunnel in background
print("\n📡 Starting Cloudflare tunnel...")
tunnel_thread = threading.Thread(target=start_cloudflare_tunnel, daemon=True)
tunnel_thread.start()

# Wait for URL
print("⏳ Waiting for tunnel to establish (10-15 seconds)...")
time.sleep(15)

print("\n" + "="*60)
print("🌐 BACKEND SERVER URL")
print("="*60)

if public_url:
    print(f"\n✅ Public URL: {public_url}")
    print(f"\n📋 IMPORTANT: Copy this URL and use it in your frontend .env file:")
    print(f"   VITE_API_BASE_URL={public_url}")
else:
    print("\n⚠️  URL not found automatically. Check the output above.")
    print("    Look for a line with: https://xxxxx.trycloudflare.com")
    print("\n📋 When you find the URL, update your frontend .env:")
    print("   VITE_API_BASE_URL=https://xxxxx.trycloudflare.com")

print("\n✅ Cloudflare tunnel is active!")
print("⚠️  Keep this notebook running!")
print("="*60)


🌐 Setting up Cloudflare Tunnel...

📡 Starting Cloudflare tunnel...
⏳ Waiting for tunnel to establish (10-15 seconds)...
2025-12-01T02:05:33Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-01T02:05:33Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-01T02:05:36Z INF +--------------------------------------------------------------------------------------------+
2025-12-01T02:05:36Z INF |  Your quick Tunnel has been created! Visit it at (it m

## 🚀 Step 5: Start Backend Server

**⚠️ CRITICAL: Keep this cell running! This is your server!**

Don't stop this cell or the server will shut down and video summarization won't work.


In [ ]:
# Import and start the Flask app
import sys
import os

# Check if files exist
if not os.path.exists('/content/app.py'):
    print("❌ ERROR: app.py not found!")
    print("\n📋 Please upload app.py first:")
    print("   1. Click the folder icon (📁) on the left sidebar")
    print("   2. Click 'Upload to session storage'")
    print("   3. Upload app.py from your local backend/ folder")
    print("   4. Re-run this cell")
    raise FileNotFoundError("app.py not found. Please upload it first.")

if not os.path.exists('/content/video_processor.py'):
    print("❌ ERROR: video_processor.py not found!")
    print("\n📋 Please upload video_processor.py first:")
    print("   1. Click the folder icon (📁) on the left sidebar")
    print("   2. Click 'Upload to session storage'")
    print("   3. Upload video_processor.py from your local backend/ folder")
    print("   4. Re-run this cell")
    raise FileNotFoundError("video_processor.py not found. Please upload it first.")

# Add /content to path
sys.path.insert(0, '/content')

try:
    # Import Flask app
    from app import app

    print("\n" + "="*60)
    print("🚀 Starting VideoWise Backend Server...")
    print("="*60)
    print(f"\n📍 Local URL: http://localhost:5000")

    # Check if we have a public URL from tunnel
    try:
        if 'public_url' in globals() and public_url and public_url != "http://localhost:5000":
            print(f"🌐 Public URL: {public_url}")
            print(f"\n📋 Copy this to frontend .env: VITE_API_BASE_URL={public_url}")
        else:
            print("\n⚠️  Public URL not set yet!")
            print("📝 Make sure you completed Step 4 (Cloudflare Tunnel)")
            print("   Then update frontend .env with the public URL")
    except:
        print("\n⚠️  Public URL not configured")
        print("📝 Complete Step 4 first (Cloudflare Tunnel)")

    print("\n✅ Server is starting...")
    print("\n⚠️  DO NOT CLOSE THIS CELL - Keep it running!")
    print("⚠️  The server will process video summarization requests")
    print("="*60)
    print("\n")

    # Run the Flask app (this will block and keep running)
    # This is the actual server - it must stay running!
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

except ImportError as e:
    print(f"❌ Import Error: {e}")
    print("\n📋 Make sure you've uploaded both files:")
    print("   - app.py")
    print("   - video_processor.py")
    print("\n   Use the file browser (📁) on the left to upload them.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 Check that:")
    print("   1. All dependencies are installed (Step 1)")
    print("   2. Files are uploaded (Step 3)")
    print("   3. API keys are set (Step 2)")


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow p

🔄 Initializing Video Processor...
Using device: cpu
Loading BLIP model...


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Initializing EasyOCR...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete✅ Groq client initialized
✅ Gemini client initialized
✅ Video Processor initialized
✅ Video processor initialized

🚀 Starting VideoWise Backend Server...

📍 Local URL: http://localhost:5000
🌐 Public URL: https://rid-accuracy-nascar-jon.trycloudflare.com

📋 Copy this to frontend .env: VITE_API_BASE_URL=https://rid-accuracy-nascar-jon.trycloudflare.com

✅ Server is starting...

⚠️  DO NOT CLOSE THIS CELL - Keep it running!
⚠️  The server will process video summarization requests


 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:20] "OPTIONS /auth/signup HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:20] "POST /auth/signup HTTP/1.1" 201 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:21] "OPTIONS /videos/list?limit=20 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:21] "OPTIONS /videos/list?limit=20 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:22] "GET /videos/list?limit=20 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:22] "GET /videos/list?limit=20 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:28] "OPTIONS /videos/list?limit=100 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:09:28] "OPTIONS /v

✅ Summary generated using Groq


INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:12] "OPTIONS /videos/status/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:13] "GET /videos/status/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:13] "OPTIONS /videos/result/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:14] "GET /videos/result/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:41] "OPTIONS /videos/status/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:41] "OPTIONS /videos/result/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:42] "GET /videos/result/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Dec/2025 02:34:42] "GET /videos/status/c676e5fb-a001-4600-8ca2-cf9b461415a5 HTTP/1.1